<a href="https://colab.research.google.com/github/e7dud7e/DAND/blob/master/L3/3_Multi_turn_chat_with_Llama_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

One of the most amazing capabilities of LLMs is to have human-like context-aware natural conversation. But LLMs are stateless, so unless the chat history is passed, LLMs won't be able to answer correctly follow up quetions. In this lesson, you'll explore how to chat with Llama 2 in a context-aware fashion.

### **Setup and helper**

In [ ]:
! pip install python-dotenv

In [ ]:
 #!echo "TOGETHER_API_KEY=<your_together_api_key>" >> .env # for local jupyter
 #!echo "TOGETHER_API_KEY=<your_together_api_key>" >> /content/.env # for google colab

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings

_ = load_dotenv(find_dotenv())
warnings.filterwarnings('ignore')

TOGETHER_API_KEY = os.environ['TOGETHER_API_KEY']

In [ ]:
import requests
import json

url = "https://api.together.xyz/inference"
headers = {
    "Authorization": f"Bearer {TOGETHER_API_KEY}",
    "Content-Type": "application/json"
}

def llama(query, add_inst=True, model="togethercomputer/llama-2-7b-chat", temperature=0.0, max_token=256):
  if add_inst:
    prompt = f"[INST]{query}[/INST]"
  else:
    prompt = query

  print(f"Prompt:\n{prompt}\n")

  data = {
      "model": model,
      "prompt": prompt,
      "temperature": temperature,
      "max_tokens": max_token
  }

  response = requests.post(url, headers=headers, json=data)
  print("Response:")

  return response.json()['output']['choices'][0]['text']

### **Provide chat history**
Store and pass previous context (prompt and response) along with new prompt, first in an intuitive way by setting a list of "User" and "Assistant" values:


In [ ]:
prompt_1 = "The typical color of a llama is what? Answer in a single word."
response_1 = "Brown"
prompt_2 = "What colors are not typical?"

prompt_chat = f"""
User: {prompt_1}
Assistant: {response_1}
User: {prompt_2}
"""
response_2 = llama(prompt_chat, model="togethercomputer/llama-2-7b-chat")
print(response_2)

Prompt: [INST]
User: The typical color of a llama is what? Answer in a single word.
Assistant: Brown
User: What colors are not typical?
[/INST]


 Sure! Here's the answer to your question:

The typical color of a llama is brown. However, some llamas can also be found in other colors such as white, gray, and even spotted or patchy patterns. So, the colors that are not typical for a llama are white, gray, and any other non-brown colors.


You can see by passing the context, Llama 2 can now correctly answer your follow up question. Let's continue the chat with another follow up question:

In [ ]:
prompt_1 = "The typical color of a llama is what? Answer in a single word."
response_1 = "Brown"
prompt_2 = "What colors are not typical?"
repoonse_2 = " Sure! Here's the answer to your question:\n\nThe typical color of a llama is brown. However, some llamas can also be found in other colors such as white, gray, and even spotted or patchy patterns. So, the colors that are not typical for a llama are white, gray, and any other non-brown colors."
prompt_3 = "Summarize the last answer with a few words."

prompt_chat = f"""
User: {prompt_1}
Assistant: {response_1}
User: {prompt_2}
Assistant: {response_2}
User: {prompt_3}
"""
response_3 = llama(prompt_chat, model="togethercomputer/llama-2-13b-chat")
print(response_3)

Prompt: [INST]
User: The typical color of a llama is what? Answer in a single word.
Assistant: Brown
User: What colors are not typical?
Assistant:  Sure! Here's the answer to your question:

The typical color of a llama is brown. However, some llamas can also be found in other colors such as white, gray, and even spotted or patchy patterns. So, the colors that are not typical for a llama are white, gray, and any other non-brown colors.
User: Summarize the last answer with a few words.
[/INST]


 Sure! Here's a summary of the last answer:

"Llamas are typically brown, but can also be white, gray, or have spotted patterns."



### **Use the recommended prompt format**

For context-aware chat, the recommended Llama 2 multi-turn prompt format is as follows - you should use the recommended format even though the previous intuitive way may work in some cases:
```
<s>[INST] {{ user_prompt_1 }} [/INST]
{{ model_response_1 }} </s>
<s>[INST] {{ user_prompt_2 }} [/INST]
{{ model_response_2 }} </s>
... ...
<s>[INST] {{ last_user_prompt }} [/INST]
```

First let's manually construct a prompt in the recommended format using the same prompts and responses:

In [ ]:
prompt_1 = "The typical color of a llama is what? Answer in a single word."
response_1 = "Brown"
prompt_2 = "What colors are not typical?"
prompt_chat = f"""
<s>[INST] {prompt_1} [/INST]
{response_1} </s>
<s>[INST] {prompt_2} [/INST]
"""

# since the prompt_chat already uses the [INST] tag, set the add_inst to False
response_2 = llama(prompt_chat, False)
print(response_2)

Prompt: 
<s>[INST] The typical color of a llama is what? Answer in a single word. [/INST]
Brown </s>
<s>[INST] What colors are not typical? [/INST]



There are several colors that are not typical for llamas, including:

* White
* Gray
* Black
* Spotted or patchy patterns
* Albino (pale pink or white skin and red eyes)
* Dilute colors (such as cream, champagne, or isabella)

It's worth noting that while these colors are not typical for llamas, they can still occur in the breed and are considered rare or unusual.


#### Helper function to format the multi-turn chats

In [ ]:
def get_prompt_chat(prompts, responses):
  formatted_prompt = f"<s>[INST] {prompts[0]} [/INST]"
  for n, response in enumerate(responses):
    prompt = prompts[n + 1]
    formatted_prompt += f"\n{response}\n </s><s>[INST] \n{ prompt }\n [/INST]"

  return formatted_prompt

You can use the helper now:

In [ ]:
prompt_1 = "The typical color of a llama is what? Answer in a single word."
response_1 = "Brown"
prompt_2 = "What colors are not typical?"

prompts = [prompt_1, prompt_2]
responses = [response_1]

response_2 = llama(get_prompt_chat(prompts, responses), False)
print(response_2)

Prompt: <s>[INST] The typical color of a llama is what? Answer in a single word. [/INST]
Brown
 </s><s>[INST] 
What colors are not typical?
 [/INST]


 While the typical color of a llama is brown, there are some colors that are not typical or less common, including:

* White
* Gray
* Black
* Cream
* Roan (a mix of white and brown)
* Palomino (a light golden color)
* Dun (a mix of red and white)

These colors are less common in llamas than the typical brown color, but they can still be found in some breeds and individuals.


For further follow up question:

In [ ]:
prompt_1 = "The typical color of a llama is what? Answer in a single word."
response_1 = "Brown"
prompt_2 = "What colors are not typical?"
repoonse_2 = " Sure! Here's the answer to your question:\n\nThe typical color of a llama is brown. However, some llamas can also be found in other colors such as white, gray, and even spotted or patchy patterns. So, the colors that are not typical for a llama are white, gray, and any other non-brown colors."
prompt_3 = "Summarize the last answer with a few words."

prompts = [prompt_1, prompt_2, prompt_3]
responses = [response_1, response_2]

response_3 = llama(get_prompt_chat(prompts, responses), False)
print(response_3)

Prompt: <s>[INST] The typical color of a llama is what? Answer in a single word. [/INST]
Brown
 </s><s>[INST] 
What colors are not typical?
 [/INST]
 While the typical color of a llama is brown, there are some colors that are not typical or less common, including:

* White
* Gray
* Black
* Cream
* Roan (a mix of white and brown)
* Palomino (a light golden color)
* Dun (a mix of red and white)

These colors are less common in llamas than the typical brown color, but they can still be found in some breeds and individuals.
 </s><s>[INST] 
Summarize the last answer with a few words.
 [/INST]


 Less common llama colors: white, gray, black, cream, roan, palomino, dun.


### **Try it out for yourself**

In [ ]:
prompt = "What is the average lifespan of a Llama? Answer the question in a few words."
print(llama(prompt))

Prompt: [INST]What is the average lifespan of a Llama? Answer the question in a few words.[/INST]


 The average lifespan of a llama is 15-20 years.


In [ ]:
# without previous context
prompt = "What animal family are they?"
print(llama(prompt))

Prompt: [INST]What animal family are they?[/INST]


 The animals that are commonly referred to as "pets" are members of various animal families. Here are some examples:

1. Dogs (Canis lupus familiaris) - They are part of the Canidae family.
2. Cats (Felis catus) - They are part of the Felidae family.
3. Birds (Aves) - There are many different species of birds that are kept as pets, including parrots, canaries, and finches.
4. Fish (Actinopterygii) - Many people keep fish as pets, including goldfish, betta fish, and tropical fish.
5. Reptiles (Testudines, Crocodilia, and Squamata) - Some popular reptile pets include turtles, bearded dragons, and geckos.
6. Amphibians (Anura) - Frogs and toads are also kept as pets.
7. Small mammals ( rodents, rabbits, guinea pigs) - These animals are often kept as pets, especially by children.
8. Hedgehogs (Erinaceus europaeus


In [ ]:
# example of multi-turn chat, with storing previous context
prompt_chat = """
User: What is the average lifespan of a Llama?
Assistant: The average lifespan of a llama is 15 to 25 years.
User: What animal family are they?

Answer the question in a few words.
"""
print(llama(prompt_chat))

Prompt: [INST]
User: What is the average lifespan of a Llama?
Assistant: The average lifespan of a llama is 15 to 25 years.
User: What animal family are they?

Answer the question in a few words.
[/INST]


 Llamas are members of the camel family (Camelidae).


In [ ]:
# use the recommended multi-turn chat format
prompts = ["What is the average lifespan of a Llama?", "What animal family are they?\n\nAnswer the question in a few words."]
responses = ["The average lifespan of a llama is 15 to 25 years."]
print(llama(get_prompt_chat(prompts, responses), False))

Prompt: <s>[INST] What is the average lifespan of a Llama? [/INST]
The average lifespan of a llama is 15 to 25 years.
 </s><s>[INST] 
What animal family are they?

Answer the question in a few words.
 [/INST]


 Llamas are members of the camel family (Camelidae).


Now that you know how to correctly query or chat with Llama 2, the next step is to understand how to best query or chat with Llama 2, covered in the next lesson.